In [ ]:
# import geopandas as gpd
# from shapely.geometry import Polygon, LineString
# import utm
# import pandas as pd
# import numpy as np
# import os

# interval_distance = 90
# buff = 350
# threshold_distance = 350

# # Path to the shapefile
# shapefile_path = r"C:\Users\magst\OneDrive\Pictures\Desktop\CONUSDATA\WFIGFirePerimeters2021\Perimeters.shp"
# #shapefile_path = r"C:\Users\magst\OneDrive\Pictures\Desktop\scratch\WINDY.shp"
# print("4")
# # Read the shapefile into a GeoDataFrame
# gdf = gpd.read_file(shapefile_path)

# # Filter for the specific perimeter
# #specific_perimeter_name = "{ACD5C207-BB83-4419-977D-3F64778081A3}"
# #gdf = gdf[gdf['poly_IRWIN'] == specific_perimeter_name]

# # Filter the GeoDataFrame for polygons with an area greater than 1000 acres
# #filtered_gdf = gdf[gdf['poly_Acres'] > 1000]
# print("5")

# # Sort the filtered GeoDataFrame by the 'poly_Acres' column in descending order
# filtered_gdf = gdf.sort_values(by='poly_Acres', ascending=False)
# print("6")

# # Input folder and POD shapefile path
# OutputFolder = "C:\\Users\\magst\\OneDrive\\Pictures\\Desktop\\FLE_SamplePoints_1\\"
# POD_shapefile_path = "C:\\Users\\magst\\OneDrive\\Pictures\\Desktop\\CONUSDATA\\FL_2021.shp"
# print("7")

# def generate_points_along_line(line, interval_distance):
#     total_length = line.length
#     num_points = int(total_length // interval_distance) + 1
#     points = [line.interpolate(i * interval_distance) for i in range(num_points)]
#     return points

# # Read the POD shapefile and filter the required lines
# test_pod = gpd.read_file(POD_shapefile_path)
# #test_pod = test_pod[test_pod['FirelineCa'].isin([1, 2, 3])]
# test_pod = test_pod[test_pod['FirelineCa'].isin([1, 2, 3]) & (test_pod['FirelineEn'] != 'Not Engaged')]

# print("8")

# for i, perimeter in enumerate(filtered_gdf.itertuples()):
#     try:
#         fire_perimeters = gpd.GeoDataFrame([perimeter], geometry='geometry')
#         print("66")
#         # Set CRS if missing
#         if fire_perimeters.crs is None:
#             fire_perimeters.set_crs(epsg=4326, inplace=True)
#         print("77")
#         lon, lat = fire_perimeters.geometry.iloc[0].centroid.coords[0]

#         centroid_lon = lon
#         centroid_lat = lat

#         utm_zone = utm.from_latlon(lat, lon)[2]
#         utm_crs = f'EPSG:326{utm_zone}'
#         fire_perimeters = fire_perimeters.to_crs(utm_crs)
#         fire_perimeters['geometry'] = fire_perimeters['geometry'].buffer(0)
#         print("88")
#         outward_buffer = fire_perimeters.geometry.buffer(buff)
#         inward_buffer = fire_perimeters.geometry.buffer(-buff)
#         inward_buffer_BREACH = fire_perimeters.geometry.buffer(-buff - 300)
#         combined_buffers = outward_buffer.difference(inward_buffer)
#         buffered_gdf = gpd.GeoDataFrame(geometry=combined_buffers, crs=fire_perimeters.crs)

#         # Transform the test_pod to match the CRS of the buffered_gdf
#         test_pod_transformed = test_pod.to_crs(buffered_gdf.crs)
#         print("99")
#         buffered_fire_perimeters = fire_perimeters.buffer(buff)
#         test_pod_clipped = gpd.clip(test_pod_transformed, buffered_fire_perimeters)
#         print("1")

#         held_pod = gpd.clip(buffered_gdf, test_pod_clipped)
#         print(f"Number of held POD geometries: {len(held_pod)}")
#         if held_pod.empty:
#             print(f"No held POD for {perimeter.poly_Incid}")
#             continue

#         held_pod['label'] = 'held_POD'
#         print("2")

#         breached_pod = gpd.clip(inward_buffer_BREACH, test_pod_clipped)
#         print(f"Number of breached POD geometries: {len(breached_pod)}")
#         if breached_pod.empty:
#             print(f"No breached POD for {perimeter.poly_Incid}")
#             continue

#         breached_pod_gdf = gpd.GeoDataFrame(geometry=breached_pod, crs=buffered_gdf.crs)
#         breached_pod_gdf['label'] = 'breached_POD'
#         print("3")

#         held_pod_points = generate_points_along_line(held_pod.geometry.iloc[0], interval_distance)
#         print("3.5")
#         breached_pod_points = generate_points_along_line(breached_pod_gdf.geometry.iloc[0], interval_distance)
#         print("4")

#         fire_perimeter_points = []
#         for boundary in fire_perimeters.boundary:
#             fire_perimeter_points.extend(generate_points_along_line(boundary, interval_distance))

#         held_pod_df = gpd.GeoDataFrame(geometry=held_pod_points)
#         breached_pod_df = gpd.GeoDataFrame(geometry=breached_pod_points)
#         fire_perimeter_df = gpd.GeoDataFrame(geometry=fire_perimeter_points)

#         merged_df = pd.concat([held_pod_df, breached_pod_df, fire_perimeter_df])

#         held_labels = [1] * len(held_pod_points)
#         breached_labels = [0] * len(breached_pod_points)
#         fire_labels = [2] * len(fire_perimeter_points)
#         labels = held_labels + breached_labels + fire_labels
#         print("5")

#         data = {
#             'labels': labels,
#             'merged_geo_series_x': merged_df.geometry.x,
#             'merged_geo_series_y': merged_df.geometry.y
#         }

#         df = pd.DataFrame(data)
#         df['labels'] = df['labels'].astype(np.int16)
#         df['merged_geo_series_x'] = df['merged_geo_series_x'].astype(np.float32)
#         df['merged_geo_series_y'] = df['merged_geo_series_y'].astype(np.float32)

#         held_and_breached_points = pd.concat([df[df['labels'] == 1], df[df['labels'] == 0]])
#         held_and_breached_gdf = gpd.GeoDataFrame(
#             held_and_breached_points,
#             geometry=gpd.points_from_xy(held_and_breached_points.merged_geo_series_x, held_and_breached_points.merged_geo_series_y)
#         ).set_crs(utm_crs)

#         held_and_breached_gdf = gpd.sjoin_nearest(held_and_breached_gdf, test_pod_clipped[['geometry', 'FirelineCa']], how='left', distance_col='distance')

#         print(f"Centroid coordinates: Latitude = {centroid_lat}, Longitude = {centroid_lon}")
#         print(held_and_breached_gdf.head())

#         unique_counts = held_and_breached_gdf['labels'].value_counts()
#         print(unique_counts)

#         held_and_breached_gdf['CatLabels'] = held_and_breached_gdf['FirelineCa'].astype(str) + held_and_breached_gdf['labels'].astype(str)
#         unique_counts = held_and_breached_gdf['CatLabels'].value_counts()
#         print(unique_counts)

#         # Add the BreachHeld column
#         #held_and_breached_gdf['BreachHeld'] = np.where(held_and_breached_gdf['labels'] == 1, 1, 0)

#         # Transform to WGS 84 before saving
#         held_and_breached_gdf_wgs84 = held_and_breached_gdf.to_crs(epsg=4326)

#         # Generate the filename from the poly_Incid attribute
#         poly_incid = perimeter.poly_Incid.replace(" ", "_").replace("(", "").replace(")", "")

#         output_shapefile_path = os.path.join(OutputFolder, f"{poly_incid}_{interval_distance}m.shp")

#         # Save the GeoDataFrame to a shapefile
#         held_and_breached_gdf_wgs84.to_file(output_shapefile_path, driver='ESRI Shapefile')
#         print(f'GeoDataFrame saved to {output_shapefile_path}')

#     except Exception as e:
#         print(f"Error processing perimeter {perimeter.poly_Incid}: {e}")

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from shapely.geometry import Polygon, LineString
import utm

# Constants
interval_distance = 90
buff = 350
threshold_distance = 350

# Paths to shapefiles
shapefile_path = r"C:\Users\magst\OneDrive\Pictures\Desktop\CONUSDATA\WFIGFirePerimeters2021\Perimeters.shp"
OutputFolder = r"C:\Users\magst\OneDrive\Pictures\Desktop\FLE_SamplePoints_1\\"
POD_shapefile_path = r"C:\Users\magst\OneDrive\Pictures\Desktop\CONUSDATA\FL_2021.shp"

# Function to generate points along a line at a specified interval
def generate_points_along_line(line, interval_distance):
    total_length = line.length
    num_points = int(total_length // interval_distance) + 1
    return [line.interpolate(i * interval_distance) for i in range(num_points)]

# Read the fire perimeter shapefile
gdf = gpd.read_file(shapefile_path)
filtered_gdf = gdf.sort_values(by='poly_Acres', ascending=False)  # Sort by polygon area (descending)

# Read and filter the POD shapefile for required fireline categories
test_pod = gpd.read_file(POD_shapefile_path)
test_pod = test_pod[test_pod['FirelineCa'].isin([1, 2, 3]) & (test_pod['FirelineEn'] != 'Not Engaged')]

# Main processing loop for each perimeter
for i, perimeter in enumerate(filtered_gdf.itertuples()):
    try:
        fire_perimeter_gdf = gpd.GeoDataFrame([perimeter], geometry='geometry')

        # Set CRS if missing
        if fire_perimeter_gdf.crs is None:
            fire_perimeter_gdf.set_crs(epsg=4326, inplace=True)

        # Convert perimeter coordinates to UTM
        lon, lat = fire_perimeter_gdf.geometry.iloc[0].centroid.coords[0]
        utm_zone = utm.from_latlon(lat, lon)[2]
        utm_crs = f'EPSG:326{utm_zone}'
        fire_perimeter_gdf = fire_perimeter_gdf.to_crs(utm_crs)

        # Buffering operations for held and breached PODs
        outward_buffer = fire_perimeter_gdf.geometry.buffer(buff)
        inward_buffer = fire_perimeter_gdf.geometry.buffer(-buff)
        inward_buffer_breach = fire_perimeter_gdf.geometry.buffer(-buff - 300)
        combined_buffers = outward_buffer.difference(inward_buffer)
        buffered_gdf = gpd.GeoDataFrame(geometry=combined_buffers, crs=fire_perimeter_gdf.crs)

        # Clip PODs using buffered fire perimeters
        test_pod_transformed = test_pod.to_crs(buffered_gdf.crs)
        buffered_fire_perimeter = fire_perimeter_gdf.buffer(buff)
        test_pod_clipped = gpd.clip(test_pod_transformed, buffered_fire_perimeter)

        # Identify held and breached PODs
        held_pod = gpd.clip(buffered_gdf, test_pod_clipped)
        if held_pod.empty:
            print(f"No held POD for {perimeter.poly_Incid}")
            continue

        breached_pod = gpd.clip(inward_buffer_breach, test_pod_clipped)
        if breached_pod.empty:
            print(f"No breached POD for {perimeter.poly_Incid}")
            continue

        # Generate points along held and breached PODs
        held_pod_points = generate_points_along_line(held_pod.geometry.iloc[0], interval_distance)
        breached_pod_points = generate_points_along_line(breached_pod.geometry.iloc[0], interval_distance)

        # Generate points along fire perimeter boundaries
        fire_perimeter_points = []
        for boundary in fire_perimeter_gdf.boundary:
            fire_perimeter_points.extend(generate_points_along_line(boundary, interval_distance))

        # Combine held, breached, and fire perimeter points into a GeoDataFrame
        held_labels = [1] * len(held_pod_points)
        breached_labels = [0] * len(breached_pod_points)
        fire_labels = [2] * len(fire_perimeter_points)
        all_labels = held_labels + breached_labels + fire_labels

        merged_df = pd.concat([
            gpd.GeoDataFrame(geometry=held_pod_points),
            gpd.GeoDataFrame(geometry=breached_pod_points),
            gpd.GeoDataFrame(geometry=fire_perimeter_points)
        ])

        # Create DataFrame with labels
        data = {
            'labels': all_labels,
            'x': merged_df.geometry.x,
            'y': merged_df.geometry.y
        }

        df = pd.DataFrame(data)

        # Create GeoDataFrame with CRS
        held_and_breached_gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df['x'], df['y']),
            crs=utm_crs
        )

        # Spatial join with nearest FirelineCa from clipped PODs
        held_and_breached_gdf = gpd.sjoin_nearest(held_and_breached_gdf, test_pod_clipped[['geometry', 'FirelineCa']], how='left', distance_col='distance')

        # Print and analyze label counts
        print(held_and_breached_gdf['labels'].value_counts())

        # Generate categorical labels
        held_and_breached_gdf['CatLabels'] = held_and_breached_gdf['FirelineCa'].astype(str) + held_and_breached_gdf['labels'].astype(str)
        print(held_and_breached_gdf['CatLabels'].value_counts())

        # Transform to WGS 84 before saving
        held_and_breached_gdf_wgs84 = held_and_breached_gdf.to_crs(epsg=4326)

        # Save the GeoDataFrame to a shapefile
        poly_incid = perimeter.poly_Incid.replace(" ", "_").replace("(", "").replace(")", "")
        output_shapefile_path = os.path.join(OutputFolder, f"{poly_incid}_{interval_distance}m.shp")
        held_and_breached_gdf_wgs84.to_file(output_shapefile_path, driver='ESRI Shapefile')

        print(f'GeoDataFrame saved to {output_shapefile_path}')

    except Exception as e:
        print(f"Error processing perimeter {perimeter.poly_Incid}: {e}")


In [ ]:
# This combines all individual shapefiles into one to make one complete dataset
import geopandas as gpd
import os
import glob
import pandas as pd

# Define the directory containing the shapefiles
directory = r"C:\Users\magst\OneDrive\Pictures\Desktop\FLE_SamplePoints_1"

# Find all the shapefiles in the directory
shapefiles = glob.glob(os.path.join(directory, "*.shp"))

# Read and combine all shapefiles into one GeoDataFrame
gdf_list = [gpd.read_file(shp) for shp in shapefiles]
combined_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

# Save the combined GeoDataFrame as a new shapefile in the same directory
output_path = os.path.join(directory, "combined.shp")
combined_gdf.to_file(output_path)

print(f"Combined shapefile saved to: {output_path}")


In [ ]:
import geopandas as gpd
import rasterio
import pandas as pd
from tqdm import tqdm  # Progress bar library
from rasterio.vrt import WarpedVRT
from rasterio import sample

def extract_multi_values_to_points(points_shapefile, raster_files, reference_raster, output_csv=None):
    # Step 1: Load the point shapefile using GeoPandas
    points_gdf = gpd.read_file(points_shapefile)

    # Step 2: Load the reference raster to get the correct CRS
    with rasterio.open(reference_raster) as ref_src:
        target_crs = ref_src.crs

    # Step 3: Reproject point geometries if necessary
    if points_gdf.crs != target_crs:
        points_gdf = points_gdf.to_crs(target_crs)
    
    # Initialize a DataFrame to store the extracted values, including original attributes
    extracted_values = points_gdf.copy()

    # Convert point geometries to a list of coordinate tuples (x, y)
    point_coords = [(geom.x, geom.y) for geom in points_gdf.geometry]

    # Step 4: Loop through each raster file and extract the values at the point locations
    for raster_path in tqdm(raster_files, desc="Processing Rasters"):
        with rasterio.open(raster_path) as src:
            # Handle CRS reprojection if necessary
            if src.crs != target_crs:
                vrt_options = {
                    'crs': target_crs,
                    'resampling': rasterio.enums.Resampling.nearest
                }
                with WarpedVRT(src, **vrt_options) as vrt:
                    values = [val[0] for val in sample.sample_gen(vrt, point_coords)]
            else:
                # Extract the raster values at all point locations in one go
                values = [val[0] for val in sample.sample_gen(src, point_coords)]
        
        # Get the raster filename to use as a column name
        column_name = raster_path.split("\\")[-1].replace('.tif', '')
        extracted_values[column_name] = values
    
    # Step 5: Optionally save to CSV
    if output_csv:
        extracted_values.to_csv(output_csv, index=False)
        print(f"Extracted values saved to: {output_csv}")
    
    return extracted_values

# Example usage:
points_shapefile = r"C:\Users\magst\OneDrive\Pictures\Desktop\FLE_SamplePoints_1\combined.shp"
reference_raster = r"C:\Users\magst\OneDrive\Pictures\Desktop\HFTCOMPELTEDATASET\US_140_PRS\US_140_PRS\Tif\us_140prs.tif"

raster_files = [
    r"C:\Users\magst\OneDrive\Pictures\Desktop\CONUSDATA\SDI\SDI_2021_CONUS\2021_90th_uphill_x100_CONUS\CONUS_2021_90_SDI.tif",
    r"C:\Users\magst\OneDrive\Pictures\Desktop\CONUSDATA\LF\LF2020_Asp_220_CONUS\Tif\LC20_Asp_220.tif",
    r"C:\Users\magst\OneDrive\Pictures\Desktop\CONUSDATA\LF\LF2020_SlpD_220_CONUS\Tif\LC20_SlpD_220.tif"
]

output_csv = r"C:\Users\magst\OneDrive\Pictures\Desktop\FLE_SamplePoints_1\extracted_valuesfull_FLE.csv"

# Call the function
extracted_df = extract_multi_values_to_points(points_shapefile, raster_files, reference_raster, output_csv)
print("done")




In [15]:
import geopandas as gpd
import os

# Define the directory and output path
output_csv = r"C:\Users\magst\OneDrive\Pictures\Desktop\FLE_SamplePoints_1\extracted_valuesfull_FLE.csv"

# Read the CSV into a GeoDataFrame
df_filtered = gpd.read_file(output_csv)

# Filter the DataFrame based on specific 'CatLabels'
df_filtered = df_filtered[df_filtered['CatLabels'].isin(['1.01', '1.00'])]

# Print the unique labels
print("Unique CatLabels:", df_filtered['CatLabels'].unique())

# Print the number of occurrences for each label
label_counts = df_filtered['CatLabels'].value_counts()
print("\nNumber of occurrences for each CatLabel:")
print(label_counts)

# Find the label with fewer occurrences
min_label = label_counts.idxmin()
min_count = label_counts.min()

# Subsample the label with more occurrences to match the label with fewer occurrences
df_more = df_filtered[df_filtered['CatLabels'] != min_label]
df_less = df_filtered[df_filtered['CatLabels'] == min_label]

# Randomly subsample the one with more occurrences to match the count of the lesser label
df_more_subsampled = df_more.sample(n=min_count, random_state=42)

# Combine the subsampled dataframes
df_filtered = pd.concat([df_more_subsampled, df_less])

# Print the new distribution
new_label_counts = df_filtered['CatLabels'].value_counts()
print("\nNew number of occurrences after subsampling:")
print(new_label_counts)


Unique CatLabels: ['1.01' '1.00']

Number of occurrences for each CatLabel:
CatLabels
1.01    45651
1.00    26889
Name: count, dtype: int64

New number of occurrences after subsampling:
CatLabels
1.01    26889
1.00    26889
Name: count, dtype: int64


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score
import numpy as np
from keras import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

# Assuming df_filtered is already loaded and contains the relevant columns and 'Treatment_Label'

# Define the features 
X = df_filtered[['CONUS_2021_90_SDI', 'LC20_Asp_220', 'LC20_SlpD_220']]

# Define the labels
y = df_filtered['CatLabels']

# One-hot encode the target
one_hot_encoder = OneHotEncoder(sparse_output=False)
y_encoded = one_hot_encoder.fit_transform(y.values.reshape(-1, 1))
# Print class names
class_names = one_hot_encoder.categories_[0]
print(f"Class Names: {class_names}")


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# MinMax scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(2, activation='softmax')) # two output classes 
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy', AUC(name='auc')])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=5, batch_size=64, validation_data=(X_test_scaled, y_test))

# Evaluate the model
loss, accuracy, auc = model.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {accuracy}')
print(f'Test AUC: {auc}')


Class Names: ['1.00' '1.01']
Epoch 1/5


C:\Users\magst\anaconda3\envs\fire1\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


589/589 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5545 - auc: 0.5686 - loss: 0.6867 - val_accuracy: 0.5739 - val_auc: 0.6232 - val_loss: 0.6725
Epoch 2/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5882 - auc: 0.6354 - loss: 0.6684 - val_accuracy: 0.6008 - val_auc: 0.6569 - val_loss: 0.6593
Epoch 3/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6029 - auc: 0.6515 - loss: 0.6601 - val_accuracy: 0.6134 - val_auc: 0.6608 - val_loss: 0.6548
Epoch 4/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6141 - auc: 0.6603 - loss: 0.6549 - val_accuracy: 0.6312 - val_auc: 0.6676 - val_loss: 0.6522
Epoch 5/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6237 - auc: 0.6666 - loss: 0.6520 - val_accuracy: 0.6264 - val_auc: 0.6680 - val_loss: 0.6520
505/505 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6238 - auc: 0.6672 - loss: 0.6519
Test Accuracy: 0.6263790726661682
Test AUC: 0.6680361032485962


In [17]:
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
from rasterio.vrt import WarpedVRT
from joblib import Parallel, delayed
from rasterio.features import rasterize

# List of raster values (you can modify or extend this as needed)
raster_values = [1]

# Function to process a single raster and extract values
def process_raster(raster_path, point_coords, target_crs):
    with rasterio.open(raster_path) as src:
        if src.crs != target_crs:
            vrt_options = {'crs': target_crs, 'resampling': rasterio.enums.Resampling.nearest}
            with WarpedVRT(src, **vrt_options) as vrt:
                values = np.array(list(vrt.sample(point_coords))).flatten()
        else:
            values = np.array(list(src.sample(point_coords))).flatten()
    return values

# Main function to extract multiple raster values for points and save to CSV
def extract_multi_values_to_points(points_shapefile, raster_files, reference_raster, output_csv, n_jobs=4):
    # Load the point shapefile using GeoPandas
    points_gdf = gpd.read_file(points_shapefile)

    # Load the reference raster to get the correct CRS
    with rasterio.open(reference_raster) as ref_src:
        target_crs = ref_src.crs

    # Reproject point geometries if necessary
    if points_gdf.crs != target_crs:
        points_gdf = points_gdf.to_crs(target_crs)

    # Initialize an empty DataFrame to store the extracted values
    extracted_values = pd.DataFrame()
    extracted_values['geometry'] = points_gdf['geometry']

    # Convert point geometries to a list of coordinate tuples (x, y)
    point_coords = [(geom.x, geom.y) for geom in points_gdf.geometry]

    # Parallelize raster extraction using joblib
    extracted_values_list = Parallel(n_jobs=n_jobs)(delayed(process_raster)(raster_path, point_coords, target_crs) for raster_path in tqdm(raster_files, desc="Processing Rasters"))

    # Convert the list of arrays to DataFrame columns
    for idx, values in enumerate(extracted_values_list):
        column_name = raster_files[idx].split("\\")[-1].replace('.tif', '')
        extracted_values[column_name] = values

    # Save to CSV
    extracted_values.to_csv(output_csv, index=False)
    print(f"Extracted values saved to: {output_csv}")

    return extracted_values

# Rasterize the probability predictions for each raster value
def rasterize_probabilities(extracted_df, softmax_predictions, reference_raster, output_prefix):
    with rasterio.open(reference_raster) as src:
        transform = src.transform
        crs = src.crs
        width = src.width
        height = src.height

    nodata_value = -9999  # Set NoData value

    for i in range(softmax_predictions.shape[1]):  # Loop over each class
        class_predictions = softmax_predictions[:, i]

        # Rasterize the point predictions within the raster grid
        prob_raster = rasterize(
            ((geom, value) for geom, value in zip(extracted_df['geometry'], class_predictions)),
            out_shape=(height, width),
            transform=transform,
            fill=nodata_value,  # Set NoData value as fill
            all_touched=True,
            dtype='float32'
        )
        
        # Mask NoData cells
        prob_raster = np.ma.masked_equal(prob_raster, nodata_value)

        output_tif = f"{output_prefix}_class_{i+1}.tif"

        with rasterio.open(
            output_tif, 'w',
            driver='GTiff',
            height=height,
            width=width,
            count=1,
            dtype='float32',
            crs=crs,
            transform=transform,
            nodata=nodata_value  # Set NoData value in the output
        ) as dst:
            dst.write(prob_raster.filled(nodata_value), 1)  # Write masked array, filling NoData

        print(f"Probability raster for class {i+1} saved to {output_tif}")

# Loop through each raster value and extract data
for i in range(len(raster_values)):
    points_shapefile = f"C:\\Users\\magst\\OneDrive\\Pictures\\Desktop\\HFTCOMPELTEDATASET\\Selected_NF_{raster_values[i]}_points.shp"
    reference_raster = r"C:\Users\magst\OneDrive\Pictures\Desktop\HFTCOMPELTEDATASET\NatForest_CO_Name.tif"
    
    # List of raster files for extraction
    raster_files = [
        r"C:\Users\magst\OneDrive\Pictures\Desktop\CONUSDATA\SDI\SDI_2021_CONUS\2021_90th_uphill_x100_CONUS\CONUS_2021_90_SDI.tif",
        r"C:\Users\magst\OneDrive\Pictures\Desktop\CONUSDATA\LF\LF2020_Asp_220_CONUS\Tif\LC20_Asp_220.tif",
        r"C:\Users\magst\OneDrive\Pictures\Desktop\CONUSDATA\LF\LF2020_SlpD_220_CONUS\Tif\LC20_SlpD_220.tif"
    ]
    
    # Output CSV path for each iteration
    output_csv = f"C:\\Users\\magst\\OneDrive\\Pictures\\Desktop\\HFTCOMPELTEDATASET\\extracted_model_values_no_filter_{raster_values[i]}.csv"
    
    # Extract raster values for current points shapefile
    extracted_df = extract_multi_values_to_points(points_shapefile, raster_files, reference_raster, output_csv, n_jobs=4)
    
    # Step 2: Prepare features for prediction if the DataFrame is not empty
    if not extracted_df.empty:
        features = extracted_df.drop(columns=['geometry'])
        scaled_features = scaler.transform(features)  # Assuming 'scaler' is pre-fitted
    
        # Step 3: Predict using the pre-trained model
        softmax_predictions = model.predict(scaled_features)
    
        # Step 4: Rasterize and save the probability rasters
        output_prefix = f"C:\\Users\\magst\\OneDrive\\Pictures\\Desktop\\FLE_SamplePoints_1\\predicted_prob_no_filterCLASS1_{raster_values[i]}"
        rasterize_probabilities(extracted_df, softmax_predictions, reference_raster, output_prefix)

print("Processing and rasterization completed for all raster values.")


Processing Rasters: 100%|████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


Extracted values saved to: C:\Users\magst\OneDrive\Pictures\Desktop\HFTCOMPELTEDATASET\extracted_model_values_no_filter_1.csv
3820/3820 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
Probability raster for class 1 saved to C:\Users\magst\OneDrive\Pictures\Desktop\FLE_SamplePoints_1\predicted_prob_no_filterCLASS1_1_class_1.tif
Probability raster for class 2 saved to C:\Users\magst\OneDrive\Pictures\Desktop\FLE_SamplePoints_1\predicted_prob_no_filterCLASS1_1_class_2.tif
Processing and rasterization completed for all raster values.
